In [ ]:
import numpy as np

class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best,
                                  shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone *= self.decay
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev],
                                  self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # retorno para a cidade de origem
        return path

    def pick_move(self, pheromone, dists, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dists) ** self.beta)

        norm_row = row / row.sum()
        move = np.random.choice(self.all_inds, 1, p=norm_row)[0]
        return move

In [ ]:
distances = np.array([[0.0001, 2, 2, 5],
                      [2, 0.0001, 4, 1],
                      [2, 4, 0.0001, 3],
                      [5, 1, 3, 0.0001]])
ant_colony = AntColony(distances, 3, 3, 100, 0.95, alpha=1, beta=2)
shortest_path = ant_colony.run()
print("Melhor caminho:", shortest_path)

Melhor caminho: ([(0, 1), (1, 3), (3, 2), (2, 0)], 8.0)


In [ ]:
distances = np.array([[0.0001, 5,      1,      8,      np.inf, np.inf],
                      [5,      0.0001, np.inf, 2,      np.inf, np.inf],
                      [1,      np.inf, 0.0001, np.inf, 1,      7],
                      [8,      1,      np.inf, 0.0001, np.inf, 1],
                      [np.inf, np.inf, 1,      np.inf, 0.0001, 6],
                      [np.inf, np.inf, 7,      1,      6,      0.0001]])
ant_colony = AntColony(distances, 3, 3, 100, 0.95, alpha=1, beta=2)
shortest_path = ant_colony.run()
print("Melhor caminho:", shortest_path)

Melhor caminho: ([(0, 2), (2, 4), (4, 5), (5, 3), (3, 1), (1, 0)], 15.0)
